In [3]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout, GridBox
from lib.model_lib import run_on_rfc_model
import io
import sys

We created a function that calls run_on_rfc_model function to generate results for each dependent variable

In [4]:
'''
  On button click, get the selected values from the selectors and run the model
'''
def on_button_click(b):
  old_stdout = sys.stdout
  sys.stdout = buffer = io.StringIO()
  
  # Get the selected values from the selectors
  selected_values = {column: selectors[column].value for column in selector_columns}
  print(selected_values)
  print(selectors['type_selector'].value)
  selected_values['type_selector'] = type_selector.value

  # Remove the columns that are not needed for the selected type
  columns_to_remove = ['roi', 'profit']
  match selectors['type_selector'].value:
    case 'revenue':
      columns_to_remove = columns_to_remove + ['vote_average', 'popularity']
    case 'popularity':
      columns_to_remove = columns_to_remove + ['revenue', 'vote_average']
    case 'vote_average':
      columns_to_remove = columns_to_remove + ['popularity', 'revenue']

  # Run the model
  run_on_rfc_model([selected_values], selectors['type_selector'].value, columns_to_remove)

  # Restore the original stdout
  sys.stdout = old_stdout

  # Get the captured output
  output = buffer.getvalue()
  print("Captured output:", output)

We identified the different options and the columns to use for field values

In [5]:
# Create the selectors
type_selector_options = ['', 'revenue', 'popularity', 'vote_average']
selector_columns = ['budget', 'runtime', 'genres_0_name', 'genres_1_name', 'genres_2_name']
genre_options = ['', 'Animation', 'Action', 'Science Fiction', 'Horror', 'Comedy', 'Drama', 'Romance', 'Fantasy', 'Thriller', 'Crime', 'TV Movie', 'Family', 'Adventure', 'War', 'Mystery', 'Documentary', 'History', 'Western', 'Music']

We added the different fields and to the widget to display.

In [ ]:
# Create the selectors, fields and button
selectors = {}
for column in selector_columns:
  if 'genres' in column:
    genre_index = int(column.split('_')[1]) + 1
    selectors[column] = widgets.Dropdown(
      options=genre_options,
      description=f'Genre {genre_index}',
      disabled=False,
      layout=Layout(width='200px'),
      id=column
    )
  else:
    selectors[column] = widgets.FloatText(
      description=column.capitalize(),
      disabled=False,
      layout=Layout(width='200px'),
      id=column
    )

type_selector = widgets.Dropdown(
  options=type_selector_options,
  description='Type Selector',
  disabled=False,
  layout=Layout(width='200px')
)

type_selector.options = [(option.replace('_', ' ').capitalize(), option) for option in type_selector_options]
selectors['type_selector'] = type_selector

predict_button = widgets.Button(
  description='Predict',
  disabled=False,
  button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
  tooltip='Click to predict',
  icon='check'  # (FontAwesome names without the `fa-` prefix)
)

# Add the button click event
predict_button.on_click(on_button_click)

# Display the selectors and button
grid = GridBox(
  children= [type_selector] + [selectors[column] for column in selector_columns] + [predict_button],
  layout=Layout(
    width='100%',
    grid_template_columns='repeat(3, 200px)',
    grid_gap='10px 10px'
  )
)

# Display the grid
display(grid)

# Get the selected values from the selectors
selected_values = {column: selectors[column].value for column in selector_columns}